# Range Data Fix
The Perovskite Database shouldn't have any ranges for numerical values (i.e. `'1.0-2.0'`). The description of the Database states any uncertainty should be averaged. Not all entered data follows this rule so we must manually replace these ranges.

In [1]:
from scripts import *

Data utils loaded


In [2]:
# Threshold (%) of data for a feature to be included
THRESHOLD = 0.50
# Depth Threshold (%) of data included from layer data.
# --- picks the maximum depth that includes the threshold % of devices
DEPTH_THRESHOLD = 0.75

TARGET_COL = "JV_default_PCE"

In [3]:
data, selector = preprocess_data(
    THRESHOLD,
    DEPTH_THRESHOLD,
    exclude_sections=[
      "Reference information",
      "Cell definition",
      "Outdoor testing",
    #   "Additional layers",
      "JV data"
      ],
    exclude_cols=[
      "Outdoor_time_start",
      "Outdoor_time_end"
      ],
    verbose=False
)

c:\Users\Violet\Documents\GitHub\PerovskiteML\src\notebooks
Loading Perovskite Data...
../../data/Perovskite_database.csv
Loading Reference Data...
../../data/pdp_units_data.xlsx
Data Initialized.
Preprocessing Data...
Threshold: 0.5, Depth: 0.75
Data Preprocessed.


In [4]:
data[selector['categorical']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43252 entries, 0 to 43251
Data columns (total 27 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   Module                                                   43242 non-null  object
 1   Module_JV_data_recalculated_per_cell                     43242 non-null  object
 2   Perovskite_single_crystal                                43242 non-null  object
 3   Perovskite_dimension_0D                                  43242 non-null  object
 4   Perovskite_dimension_2D                                  43242 non-null  object
 5   Perovskite_dimension_2D3D_mixture                        43242 non-null  object
 6   Perovskite_dimension_3D                                  43252 non-null  bool  
 7   Perovskite_dimension_3D_with_2D_capping_layer            43242 non-null  object
 8   Perovskite_composition_perovskite_AB

In [5]:
data[selector['numerical']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43252 entries, 0 to 43251
Data columns (total 4 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Module_number_of_cells_in_module                  43242 non-null  float64
 1   Perovskite_deposition_number_of_deposition_steps  43242 non-null  float64
 2   Stability_average_over_n_number_of_cells          43242 non-null  float64
 3   Stability_flexible_cell_number_of_bending_cycles  43242 non-null  float64
dtypes: float64(4)
memory usage: 1.3 MB


In [6]:
data[selector['patterned']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43252 entries, 0 to 43251
Data columns (total 34 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   Substrate_stack_sequence_Layer_0                                   42848 non-null  object
 1   Substrate_stack_sequence_Layer_1                                   42670 non-null  object
 2   ETL_stack_sequence_Layer_0                                         42831 non-null  object
 3   ETL_stack_sequence_Layer_1                                         26265 non-null  object
 4   ETL_deposition_procedure_Layer_0                                   42035 non-null  object
 5   ETL_deposition_procedure_Layer_1                                   25808 non-null  object
 6   Perovskite_dimension_list_of_layers                                42757 non-null  object
 7   Perovskite_composition_a_ions_F

In [7]:
data[selector['patterned']].apply(pd.to_numeric, errors='ignore').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43252 entries, 0 to 43251
Data columns (total 34 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Substrate_stack_sequence_Layer_0                                   42848 non-null  object 
 1   Substrate_stack_sequence_Layer_1                                   42670 non-null  object 
 2   ETL_stack_sequence_Layer_0                                         42831 non-null  object 
 3   ETL_stack_sequence_Layer_1                                         26265 non-null  object 
 4   ETL_deposition_procedure_Layer_0                                   42035 non-null  object 
 5   ETL_deposition_procedure_Layer_1                                   25808 non-null  object 
 6   Perovskite_dimension_list_of_layers                                42757 non-null  float64
 7   Perovskite_composition

C:\Users\Violet\AppData\Local\Temp\ipykernel_19824\1636053075.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data[selector['patterned']].apply(pd.to_numeric, errors='ignore').info()


In [8]:
import re
pattern = r'^\d+(\.\d+)?\s*-\s*\d+(\.\d+)?$'

In [9]:
col_name = 'Backcontact_thickness_list'
mask = data[col_name].notna()
col = data[col_name][mask]
mask = col.str.contains(pattern, regex=True)
col[mask]

C:\Users\Violet\AppData\Local\Temp\ipykernel_19824\4203228665.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = col.str.contains(pattern, regex=True)


Series([], Name: Backcontact_thickness_list, dtype: object)

In [10]:
def split_avg(x):
    return np.float64(x.replace(" ", "").split('-')).mean()

replacements = []
for col_name in DATASET.data.columns:
    if 'Ref' in col_name:
        print(f'skip {col_name}')
        continue
    col = DATASET.data[col_name].apply(str)
    try: 
        mask = col.str.contains(pattern, regex=True)
        x = col[mask].apply(split_avg)
        print(x)
        if len(x) > 0:
            replacements.append((col_name, mask, x))
        print(DATASET.data[col_name][mask])

    except:
        None

C:\Users\Violet\AppData\Local\Temp\ipykernel_19824\554148533.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = col.str.contains(pattern, regex=True)


Series([], Name: Unnamed: 0.1, dtype: object)
Series([], Name: Unnamed: 0.1, dtype: int64)
Series([], Name: Unnamed: 0, dtype: object)
Series([], Name: Unnamed: 0, dtype: int64)
skip Ref_ID
skip Ref_ID_temp
skip Ref_name_of_person_entering_the_data
skip Ref_data_entered_by_author
skip Ref_DOI_number
skip Ref_lead_author
skip Ref_publication_date
skip Ref_journal
skip Ref_part_of_initial_dataset
skip Ref_original_filename_data_upload
skip Ref_free_text_comment
skip Ref_internal_sample_id
Series([], Name: Cell_stack_sequence, dtype: object)
Series([], Name: Cell_stack_sequence, dtype: object)
Series([], Name: Cell_area_total, dtype: object)
Series([], Name: Cell_area_total, dtype: float64)
Series([], Name: Cell_area_measured, dtype: object)
Series([], Name: Cell_area_measured, dtype: float64)
Series([], Name: Cell_number_of_cells_per_substrate, dtype: object)
Series([], Name: Cell_number_of_cells_per_substrate, dtype: float64)
Series([], Name: Cell_architecture, dtype: object)
Series([],

In [27]:
for (col, mask, x) in replacements:
    DATASET.data[col][mask] = x

C:\Users\Violet\AppData\Local\Temp\ipykernel_18468\596337092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATASET.data[col][mask] = x
C:\Users\Violet\AppData\Local\Temp\ipykernel_18468\596337092.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a singl

In [30]:
DATASET.data.to_csv('Perovskite_database.csv')